In [ ]:
# load dependencies and plot formatting options
import glob
import matplotlib.pyplot as plt
import rasterio as rio
import datetime
import numpy as np
import xarray as xr
import statistics

plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

## SET USER DEFINITIONS

In [ ]:
# specify the years to process over
years = [2019,2020,2021,2022,2023]
# specify the focus year for saving purposes
year_select = 2022
# specify the index of the first PS image to use for each of the "years" above
# 0 means that DSD will be calculated using all snow cover maps
starting_array = [21,30,27,81,65]

# domain ID and file directories
DOMID = 'DPO'
data_direc = '/Users/jpflug/Documents/Projects/cubesatReanaly/Data/Meadows/'+DOMID+'/self_classified/SCA/'

## FUNCTIONS -- DO NOT EDIT

In [ ]:
# calculate the days from 1 October for each image and create the snow cover data cube (time,y,x)
def DSD_index(year,files):
    # instantiate
    data_list,daydiff = [],[]
    # set the reference date to the start of the water year
    ref_date = datetime.date(year-1,10,1)

    # loop through the snow cover observations
    for fCount,file in enumerate(files):
        # determine/log date
        breakFile = file.split('/')[-1].split('_')[0]
        dater = datetime.date(int(breakFile[0:4]),int(breakFile[4:6]),int(breakFile[6:8]))

        # determine how many days into the water year the PS obs is from
        daydiff.append((dater-ref_date).days)

        # read in and append the data
        with rio.open(file) as src:
            data = src.read(1)
            data_list.append(data)

    # create stacked dataframe for snow cover
    stacked_data = np.stack(data_list, axis=0).astype(float)
    daydiff = np.array(daydiff)
    # print(daydiff)
    return stacked_data,daydiff

# function for correcting no-snow dates preceded and followed by snow cover (SEASONAL SNOW ONLY!!)
def find_spurious_0(arr):
    locations = []
    time, x, y = arr.shape
    for t in range(1, time - 1):
        for i in range(x):
            for j in range(y):
                if arr[t-1, i, j] == 1 and arr[t, i, j] == 0 and arr[t+1, i, j] == 1:
                    locations.append((t, i, j))
    return locations
# function for correcting snow dates preceded and followed by no snow cover (SEASONAL SNOW ONLY!!)
def find_spurious_1(arr):
    locations = []
    time, x, y = arr.shape
    for t in range(1, time - 1):
        for i in range(x):
            for j in range(y):
                if arr[t-1, i, j] == 0 and arr[t, i, j] == 1 and arr[t+1, i, j] == 0:
                    locations.append((t, i, j))
    return locations
# find the last dates with consecute snow-present observations
def find_last_consecutive_ones_after_zero(arr):
    max_consecutive_indices = []
    n = len(arr)
    i = 0
    while i < n - 1:
        if arr[i] == 0 and arr[i + 1] == 1:
            start_index = i + 1
            while start_index < n - 1 and arr[start_index] == 1 and arr[start_index + 1] == 1:
                start_index += 1
            consecutive_indices = [start_index]
            if len(consecutive_indices) > len(max_consecutive_indices):
                max_consecutive_indices = consecutive_indices
            i = start_index + 1
        else:
            i += 1
    return max_consecutive_indices
# fill spurious values using the defined "fill_value"
def fill_locations(arr, locations,fill_value):
    for loc in locations:
        t, x, y = loc
        arr[t, x, y] = fill_value
    return arr
def fill_nodata(stacked_data):
    t,y,x = stacked_data.shape
    for tt in np.arange(t):
        tt_date = stacked_data[tt,:,:]
        if tt == 0:
            tt_date[tt_date == 2] = 1
        else:
            tt_ref = stacked_data[tt-1,:,:]
            tt_date[tt_date == 2] = tt_ref[tt_date == 2] 
        stacked_data[tt,:,:] = tt_date
    return stacked_data
# perform data QA and QC using the functions from above (SEASONAL SNOW ONLY!!)
def DSD_qaqc(stacked_data):
    locations_1 = find_spurious_0(stacked_data)
    stacked_data = fill_locations(stacked_data,locations_1,1)
    locations_2 = find_spurious_1(stacked_data)
    stacked_data = fill_locations(stacked_data,locations_2,0)
    _,y,x = stacked_data.shape
    for j in range(y):
        for i in range(x):
            location_3 = find_last_consecutive_ones_after_zero(stacked_data[:,j,i])
            if len(location_3) > 0:
                stacked_data[:location_3[0],j,i] = 1
    return stacked_data

def fill_DSDdates(daydiff,dsd,nodata_ref):
    # determine the actual day since October 1 the observation came from
    dsd_date = daydiff[dsd].astype(float)
    
    temp = np.empty(dsd.shape)
    # loop through the dsd array
    # determine the snow classification on the dsd date
    for j in range(dsd.shape[0]):
        for i in range(dsd.shape[1]):
            time_index = dsd[j,i]
            temp[j,i] = nodata_ref[time_index,j,i]
            
    # determine where when dsd occurred on a bad observation pixel account for that in the uncertainty
    filled_y,filled_x = np.where(temp == 2)
    if len(filled_y) > 0:
        for j,i in zip(filled_y,filled_x):
            time_index = dsd[j,i]
            temp = nodata_ref[time_index,j,i]
            while temp == 2:
                time_index = dsd[j,i] - 1
                temp = nodata_ref[time_index,j,i]
            # fill the dsd with the last good date
            dsd[j,i] = time_index
    
    # determine the snow-present day just before that
    temp = daydiff[dsd-1].astype(float)
    # and calculate the uncertainty
    temp[dsd < 0] = np.nan
    dsd_uncertain = dsd_date-temp
    # set the dsd as the halfway point
    dsd_date = np.ceil((dsd_date+temp)/2)#.astype(int)
    return dsd_date,dsd_uncertain

#### CALCULATE THE DSD AND DSD UNCERTAINTY FOR EACH PIXEL AND YEAR

In [ ]:
# loop through each testing year
for year_index,testing_year in enumerate(years):
    # load the processed snow cover files
    print('year: ',year_index)
    files = sorted(glob.glob(data_direc+str(testing_year)+'*SCA.tif'))[starting_array[year_index]:]

    # initialize
    ref_date = datetime.date(testing_year-1,10,1)
    data_list,daydiff = [],[]
    # loop through the good planet scenes
    for fCount,file in enumerate(files):
        # determine the date from the filename and append to the record
        breakFile = file.split('/')[-1].split('_')[0]
        dater = datetime.date(int(breakFile[0:4]),int(breakFile[4:6]),int(breakFile[6:8]))
        daydiff.append((dater-ref_date).days)

        # read in and store the data
        with rio.open(file) as src:
            data = src.read(1)
            data_list.append(data)

    # create a stacked data array for snow cover
    stacked_data = np.stack(data_list, axis=0).astype(float)
    daydiff = np.array(daydiff)

    # fill instances with no-data classifications (from image artifacts)
    nodata_ref = stacked_data.copy()
    stacked_data = fill_nodata(stacked_data)

    # do data filtering and post-processing (SEASONAL SNOW ONLY!!)
    stacked_data = DSD_qaqc(stacked_data)

    # save the data if this is a modeling year
    if testing_year == year_select:
        np.save(data_direc+'processed_'+str(year_select)+'/DSD_stacked.npy',stacked_data)
        np.save(data_direc+'processed_'+str(year_select)+'/DSD_daydiff.npy',daydiff)

    # calculate the date of snow disappearance
    dsd = np.argmin(stacked_data,axis=0)

    # determine the actual dsd date and uncertainty
    dsd_masked = dsd.copy()
    dsd,dsd_uncertain = fill_DSDdates(daydiff,dsd,nodata_ref)
    # mask values where snow was never observed
    dsd[dsd_masked == 0] = np.nan

    # calculate the domain annual-normalized pattern
    dsd_anomaly = dsd-np.nanmedian(dsd)

    # append the data 
    if year_index == 0:
        shpp = dsd_anomaly.shape
        dsd_saver = np.empty((len(years),shpp[0],shpp[1]))
        dsd_anomaly_saver = np.empty((len(years),shpp[0],shpp[1]))
        dsd_uncertain_saver = np.empty((len(years),shpp[0],shpp[1]))    
    dsd_saver[year_index,:,:] = dsd
    dsd_anomaly_saver[year_index,:,:] = dsd_anomaly
    dsd_uncertain_saver[year_index,:,:] = dsd_uncertain

######## save additional data, as desired ##############
# np.save(data_direc+'processed_'+str(year_select)+'/DSD.npy',dsd_saver)
# np.save(data_direc+str(year_select)+'/DSD_anomaly.npy',dsd_anomaly_saver)
# np.save(data_direc+'/DSD_uncertain_saver.npy',dsd_uncertain_saver)

# holder = xr.open_dataset(files[0])
# holder['band_data'][0,:,:] = np.mean(dsd_anomaly_saver,axis=0)
# holder['band_data'].rio.to_raster(data_direc+'averageDSD_anomaly.tif')

In [ ]:
# perform plotting for the DSD and DSD uncertainty
from mpl_toolkits.axes_grid1 import make_axes_locatable

mask = np.sum(dsd_saver,axis=0)

fg,ax = plt.subplots(2,len(years),figsize=(9,5),
                     sharex='col', sharey='row', gridspec_kw={'height_ratios': [1, 1],'hspace': 0.02, 'wspace': 0.05})

for idx,year in enumerate(years):
    data = dsd_anomaly_saver[idx,:,:]
    data[np.isnan(mask)] = np.nan
    try:
        out1 = ax[1,idx].imshow(data,vmin=-20,vmax=20,interpolation='none',cmap='RdBu')
        ax[0,idx].set_title(year)
    except:
        out1 = ax[1].imshow(data,vmin=-20,vmax=20,interpolation='none',cmap='RdBu')
        ax[0].set_title(year)
    
for idx,year in enumerate(years):
    # data = dsd_uncertain_saver[idx,:,:]
    data = dsd_saver[idx,:,:]
    data[np.isnan(mask)] = np.nan
    try:
        out2 = ax[0,idx].imshow(data,vmin=180,vmax=280,interpolation='none',cmap='inferno')
    except:
        out2 = ax[0].imshow(data,vmin=0,vmax=14,interpolation='none',cmap='Greens')
    
for axx in np.ravel(ax):
    axx.set_xticks([])
    axx.set_yticks([])
    axx.set_facecolor([0.6,0.6,0.6])
    
pos_bott = ax[1,-1].get_position()
pos_top = ax[0,-1].get_position()

cax1 = fg.add_axes([pos_bott.x1 + 0.02,pos_bott.y0,0.02,pos_top.y1-pos_bott.y0])
cbar1 = fg.colorbar(out2, cax=cax1, orientation='vertical')
cbar1.set_label('Date of snow disappearance [days from 1 Oct]',fontsize=11)
  
cax2 = fg.add_axes([pos_bott.x1 + 0.12,pos_bott.y0,0.02,pos_top.y1-pos_bott.y0])
cbar2 = fg.colorbar(out1, cax=cax2, orientation='vertical')
cbar2.set_label('Date of snow disappearance\nanomaly [days]',fontsize=11)